In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import import_ipynb
import Captcha_seg
import time

importing Jupyter notebook from Captcha_seg.ipynb


In [2]:
CATEGORIES =[]
for i in os.listdir("captcha_segmented/"):
    CATEGORIES.append(i)
print(CATEGORIES)

['6', 'q', 'F', 'E', '3', 'U', 'K', 'M', 'd', '4', 'X', 'W', 'y', 'm', 'Z', 'V', 'H', '8', 'C', 'S', 'P', 'A', 'n', 'R', '5', 'Q', 'f', 'N', 'b', '7', 'G', '2', 'Y', 'B', 'e', 'a', 'h']


In [3]:
SEGMENTS_DIR = 'Segmented'
CAPTCHA_DIR = 'test/'
predictions = []
b_captcha = []

DATADIR = "./captcha_segmented/"
#CATEGORIES= ["2","3","4","5","6","7","8","b","c","d","e","f","g","m","n","p","w","x","y"]
for category in CATEGORIES:
    path = os.path.join(DATADIR, category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
        #plt.imshow(img_array, cmap="gray")
        break
    break

In [4]:
print(img_array.shape)

(30, 16)


In [5]:
IMG_SIZE = 50

new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
#plt.imshow(new_array, cmap= 'gray')
#plt.show()

In [6]:
train_data = []

def create_train_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category) 
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                train_data.append([new_array,class_num])
            except Exception as e:
                pass
            
create_train_data()

In [7]:
print(len(train_data))

4970


In [8]:
import random

random.shuffle(train_data)

In [9]:
for sample in train_data[:10]:
    print(sample[1])

5
5
10
15
36
14
24
23
2
30


In [10]:
X = []
y = []

In [11]:
for features, label in train_data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In [12]:
import pickle

pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()
pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [13]:
pickle_in= open("X.pickle", "rb")
X = pickle.load(pickle_in)
X.shape[1:]

(50, 50, 1)

In [14]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout,Activation, Flatten, Conv2D, MaxPooling2D

X = pickle.load(open("X.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))

X = X/255.0

model = Sequential()

model.add(Conv2D(64,(3,3),input_shape= X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))


model.add(Dense(37))
model.add(Activation("softmax"))

from keras.utils import to_categorical
y_binary = to_categorical(y)

model.compile(loss="categorical_crossentropy",
             optimizer="adam",
             metrics=['accuracy'])

model.fit(X,y_binary,batch_size=128, epochs=30, validation_split = 0.2)


/home/bg/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Train on 3976 samples, validate on 994 samples
Epoch 1/30
3976/3976 [==============================] - 29s 7ms/step - loss: 3.5778 - acc: 0.0455 - val_loss: 3.5723 - val_acc: 0.0412
Epoch 2/30
3976/3976 [==============================] - 30s 8ms/step - loss: 3.5529 - acc: 0.0543 - val_loss: 3.5656 - val_acc: 0.0392
Epoch 3/30
3976/3976 [==============================] - 29s 7ms/step - loss: 3.5159 - acc: 0.0699 - val_loss: 3.4889 - val_acc: 0.0654
Epoch 4/30
3976/3976 [==============================] - 28s 7ms/step - loss: 3.4353 - acc: 0.0853 - val_loss: 3.4578 - val_acc: 0.0915
Epoch 5/30
3976/3976 [==============================] - 28s 7ms/step - loss: 3.3044 - acc: 0.1122 - val_loss: 3.2661 - val_acc: 0.1227
Epoch 6/30
3976/3976 [==============================] - 28s 7ms/step - loss: 3.0789 - acc: 0.1693 - val_loss: 3.0533 - val_acc: 0.1771
Epoch 7/30
3976/3976 [==============================] - 28s 7ms/step - loss: 2.8507 - acc: 0.2241 - val_loss: 3.0024 - val_acc: 0.1841
Epoch 8/

In [15]:
loss_value, metrics_value = model.evaluate(X,y_binary,batch_size=10)

4970/4970 [==============================] - 10s 2ms/step


In [16]:
print(loss_value, metrics_value)

1.2834256712714551 0.741649894887053


In [17]:
def similarity(a,b):
    i =0
    for x,y in zip(a,b):
        if x==y:
            i+= 1
    percentage = (i/5) * 100
    return percentage

def prepare(filepath):
    IMG_SIZE = 50
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE,IMG_SIZE))
    #plt.imshow(new_array,'gray')
    return new_array.reshape(-1, IMG_SIZE,IMG_SIZE,1)

total_time = 0
score = 0
count = 0
for captchas in os.listdir(CAPTCHA_DIR):
    start_time = time.time()
    predict_list= []
    captcha_name = captchas.split('.')
    print('\n')
    print('Actual: ',captcha_name[0])
    
    captcha_path =os.path.join(CAPTCHA_DIR,captchas)
    Captcha_seg.segment(captcha_path)
    
    files = []
    for segments in os.listdir(SEGMENTS_DIR):
        files.append(segments)
        files.sort()
    
    for segments in files:
        filepath = os.path.join(SEGMENTS_DIR,segments)
        prediction = model.predict([prepare(filepath)])
        predict_list.append(CATEGORIES[np.argmax(prediction[0])])
    
    elapsed_time = time.time()-start_time
    total_time += elapsed_time
    final_prediction = ''.join(predict_list)
    if captcha_name[0]== final_prediction:
        score +=1
    
    print('Predicted:',final_prediction,'\n')
    print('Percentage Similarity= ',similarity(captcha_name[0],final_prediction))
    print('Elapsed time= ',elapsed_time,'s')
    print('<------------------------------->')    
    predictions.append(similarity(captcha_name[0],final_prediction))
    count += 1


print('Total number of CAPTCHAS broken: ',score)    
print('Total number of CAPTCHAS attempted: ',count)
avg_prediction = score/count
print('Average Characters Predicted= ',avg_prediction,'%')
print('Total time taken= ',total_time,'s')
#model = tf.keras.models.load_model('captcha_alpha.model')
#prediction = model.predict([prepare('3.png')])
#print(len(prediction[0]))
#print(prediction[0])
#print(CATEGORIES[np.argmax(prediction[0])])



Actual:  ymVKf
Predicted: W7VWY 

Percentage Similarity=  20.0
Elapsed time=  2.041058301925659 s
<------------------------------->


Actual:  NdEnK
Predicted: NQSnX 

Percentage Similarity=  40.0
Elapsed time=  0.04696965217590332 s
<------------------------------->


Actual:  RV6BS
Predicted: RV6BQ 

Percentage Similarity=  80.0
Elapsed time=  0.034113407135009766 s
<------------------------------->


Actual:  b2Pqb
Predicted: Y4PQQ 

Percentage Similarity=  20.0
Elapsed time=  0.056443214416503906 s
<------------------------------->


Actual:  aRhdP
Predicted: aaPan 

Percentage Similarity=  20.0
Elapsed time=  0.017457962036132812 s
<------------------------------->


Actual:  MCGbF
Predicted: UYQSS 

Percentage Similarity=  0.0
Elapsed time=  0.05054736137390137 s
<------------------------------->


Actual:  CqCWP
Predicted: Qq3WX 

Percentage Similarity=  40.0
Elapsed time=  0.03596639633178711 s
<------------------------------->


Actual:  E3Aqb
Predicted: P3XWV 

Percentage S

Predicted: PVWmA 

Percentage Similarity=  20.0
Elapsed time=  0.01870894432067871 s
<------------------------------->


Actual:  ESPFZ
Predicted: ESPYS 

Percentage Similarity=  60.0
Elapsed time=  0.01588296890258789 s
<------------------------------->


Actual:  SWUFQ
Predicted: SWU4a 

Percentage Similarity=  60.0
Elapsed time=  0.01490330696105957 s
<------------------------------->


Actual:  UXWWX
Predicted: US6AW 

Percentage Similarity=  20.0
Elapsed time=  0.014078855514526367 s
<------------------------------->


Actual:  Q4CV6
Predicted: CQ4V6 

Percentage Similarity=  40.0
Elapsed time=  0.014299869537353516 s
<------------------------------->


Actual:  XYaW8
Predicted: XYaWA 

Percentage Similarity=  80.0
Elapsed time=  0.01418614387512207 s
<------------------------------->


Actual:  bBZ8S
Predicted: aQC3S 

Percentage Similarity=  20.0
Elapsed time=  0.014512300491333008 s
<------------------------------->


Actual:  BF4b3
Predicted: BF2bf 

Percentage Similarity=  60

Predicted: 6nXaZ 

Percentage Similarity=  40.0
Elapsed time=  0.023677349090576172 s
<------------------------------->


Actual:  HQZG5
Predicted: ASZG6 

Percentage Similarity=  40.0
Elapsed time=  0.021402597427368164 s
<------------------------------->


Actual:  WBFV8
Predicted: WXPVW 

Percentage Similarity=  40.0
Elapsed time=  0.019617557525634766 s
<------------------------------->


Actual:  WPd4P
Predicted: WPPQP 

Percentage Similarity=  60.0
Elapsed time=  0.01941680908203125 s
<------------------------------->


Actual:  BbMPK
Predicted: 33MPX 

Percentage Similarity=  40.0
Elapsed time=  0.01853156089782715 s
<------------------------------->


Actual:  UNfBM
Predicted: VnNKX 

Percentage Similarity=  0.0
Elapsed time=  0.02002429962158203 s
<------------------------------->


Actual:  8XU5N
Predicted: XXVaV 

Percentage Similarity=  20.0
Elapsed time=  0.0183868408203125 s
<------------------------------->


Actual:  MQUeS
Predicted: KQaWS 

Percentage Similarity=  40.0

Predicted: AC3A3 

Percentage Similarity=  0.0
Elapsed time=  0.03272414207458496 s
<------------------------------->


Actual:  EU5hn
Predicted: aWqhP 

Percentage Similarity=  20.0
Elapsed time=  0.020218610763549805 s
<------------------------------->


Actual:  ZAQUZ
Predicted: ZqQWZ 

Percentage Similarity=  60.0
Elapsed time=  0.03787970542907715 s
<------------------------------->


Actual:  Cm5fS
Predicted: CPAQS 

Percentage Similarity=  40.0
Elapsed time=  0.015810012817382812 s
<------------------------------->


Actual:  RVFbE
Predicted: FVSVS 

Percentage Similarity=  20.0
Elapsed time=  0.02611827850341797 s
<------------------------------->


Actual:  f4m5F
Predicted: fXC6P 

Percentage Similarity=  20.0
Elapsed time=  0.015484094619750977 s
<------------------------------->


Actual:  7VAUH
Predicted: 7VSVX 

Percentage Similarity=  40.0
Elapsed time=  0.024729013442993164 s
<------------------------------->


Actual:  b4GAA
Predicted: b6qSA 

Percentage Similarity=  40

Predicted: Xm6KG 

Percentage Similarity=  20.0
Elapsed time=  0.0170590877532959 s
<------------------------------->


Actual:  Z8Wma
Predicted: Z3WQa 

Percentage Similarity=  60.0
Elapsed time=  0.0185699462890625 s
<------------------------------->


Actual:  aMKyK
Predicted: VXKYW 

Percentage Similarity=  20.0
Elapsed time=  0.015130043029785156 s
<------------------------------->


Actual:  8PRKq
Predicted: SPXVa 

Percentage Similarity=  20.0
Elapsed time=  0.014879465103149414 s
<------------------------------->


Actual:  GVEZe
Predicted: GXFVS 

Percentage Similarity=  20.0
Elapsed time=  0.015530586242675781 s
<------------------------------->


Actual:  PUmUX
Predicted: PWVYX 

Percentage Similarity=  40.0
Elapsed time=  0.015930891036987305 s
<------------------------------->


Actual:  E4VnU
Predicted: PAVmU 

Percentage Similarity=  40.0
Elapsed time=  0.015720129013061523 s
<------------------------------->


Actual:  NSNFM
Predicted: W6NF8 

Percentage Similarity=  40

...

Predicted: 6VS6X 

Percentage Similarity=  20.0
Elapsed time=  0.029629945755004883 s
<------------------------------->


Actual:  ZSSWX
Predicted: SSSWX 

Percentage Similarity=  80.0
Elapsed time=  0.017611026763916016 s
<------------------------------->


Actual:  EY3CP
Predicted: EYmmP 

Percentage Similarity=  60.0
Elapsed time=  0.014408588409423828 s
<------------------------------->


Actual:  H3n3U
Predicted: H3maZ 

Percentage Similarity=  40.0
Elapsed time=  0.014729976654052734 s
<------------------------------->


Actual:  KPqeF
Predicted: KPqeh 

Percentage Similarity=  80.0
Elapsed time=  0.014755487442016602 s
<------------------------------->


Actual:  K2QUS
Predicted: aZQWS 

Percentage Similarity=  40.0
Elapsed time=  0.014743804931640625 s
<------------------------------->


Actual:  Xa3yd
Predicted: 633b6 

Percentage Similarity=  20.0
Elapsed time=  0.014436960220336914 s
<------------------------------->


Actual:  dEYFC
Predicted: aQYFC 

Percentage Similarity=

Predicted: aCX3Q 

Percentage Similarity=  40.0
Elapsed time=  0.01849842071533203 s
<------------------------------->


Actual:  mqGNK
Predicted: mqQaV 

Percentage Similarity=  40.0
Elapsed time=  0.026495695114135742 s
<------------------------------->


Actual:  NQQXn
Predicted: YQQQW 

Percentage Similarity=  40.0
Elapsed time=  0.014771699905395508 s
<------------------------------->


Actual:  mSHQF
Predicted: PWFQQ 

Percentage Similarity=  20.0
Elapsed time=  0.014513731002807617 s
<------------------------------->


Actual:  RVeVb
Predicted: XVeXC 

Percentage Similarity=  40.0
Elapsed time=  0.014501571655273438 s
<------------------------------->


Actual:  64HUW
Predicted: ZAXWW 

Percentage Similarity=  20.0
Elapsed time=  0.02204728126525879 s
<------------------------------->


Actual:  HGZhV
Predicted: H4ZmV 

Percentage Similarity=  60.0
Elapsed time=  0.0167691707611084 s
<------------------------------->


Actual:  HPqYZ
Predicted: XaqYZ 

Percentage Similarity=  60

Predicted: HVmF4 

Percentage Similarity=  20.0
Elapsed time=  0.025135517120361328 s
<------------------------------->


Actual:  RH3EM
Predicted: RY3mQ 

Percentage Similarity=  40.0
Elapsed time=  0.021526098251342773 s
<------------------------------->


Actual:  M4MUX
Predicted: VWKWW 

Percentage Similarity=  0.0
Elapsed time=  0.01918196678161621 s
<------------------------------->


Actual:  475YS
Predicted: 4Z5Sf 

Percentage Similarity=  40.0
Elapsed time=  0.020942211151123047 s
<------------------------------->


Actual:  deaSP
Predicted: S3aSP 

Percentage Similarity=  60.0
Elapsed time=  0.019768476486206055 s
<------------------------------->


Actual:  ZhyWU
Predicted: CKUVP 

Percentage Similarity=  0.0
Elapsed time=  0.028015613555908203 s
<------------------------------->


Actual:  bPCaE
Predicted: b7CaA 

Percentage Similarity=  60.0
Elapsed time=  0.03816103935241699 s
<------------------------------->


Actual:  PqC46
Predicted: PqQXC 

Percentage Similarity=  40

Predicted: XY6Va 

Percentage Similarity=  0.0
Elapsed time=  0.022905349731445312 s
<------------------------------->


Actual:  NF7P7
Predicted: Xm7P7 

Percentage Similarity=  60.0
Elapsed time=  0.021297693252563477 s
<------------------------------->


Actual:  e6dXW
Predicted: S6CWW 

Percentage Similarity=  40.0
Elapsed time=  0.018639326095581055 s
<------------------------------->


Actual:  m8QY7
Predicted: PAWPZ 

Percentage Similarity=  0.0
Elapsed time=  0.01668715476989746 s
<------------------------------->


Actual:  ZNPSV
Predicted: ZNPSW 

Percentage Similarity=  80.0
Elapsed time=  0.03455400466918945 s
<------------------------------->


Actual:  PXh5E
Predicted: PWm5E 

Percentage Similarity=  60.0
Elapsed time=  0.015540838241577148 s
<------------------------------->


Actual:  3FXBF
Predicted: aCb6P 

Percentage Similarity=  0.0
Elapsed time=  0.028719425201416016 s
<------------------------------->


Actual:  nZSQn
Predicted: QZSCP 

Percentage Similarity=  40.

Predicted: V7S78 

Percentage Similarity=  100.0
Elapsed time=  0.014983415603637695 s
<------------------------------->


Actual:  NH2dn
Predicted: NW2dQ 

Percentage Similarity=  60.0
Elapsed time=  0.017028331756591797 s
<------------------------------->


Actual:  ybmmn
Predicted: WmmmQ 

Percentage Similarity=  40.0
Elapsed time=  0.0192410945892334 s
<------------------------------->


Actual:  HC5Sf
Predicted: XCSYf 

Percentage Similarity=  40.0
Elapsed time=  0.018471240997314453 s
<------------------------------->


Actual:  W46QH
Predicted: WYX6W 

Percentage Similarity=  20.0
Elapsed time=  0.01978898048400879 s
<------------------------------->


Actual:  8VdSZ
Predicted: SXdSZ 

Percentage Similarity=  60.0
Elapsed time=  0.015032529830932617 s
<------------------------------->


Actual:  dX4Mf
Predicted: AX4XP 

Percentage Similarity=  40.0
Elapsed time=  0.01496267318725586 s
<------------------------------->


Actual:  H4PFe
Predicted: W4PEm 

Percentage Similarity=  4